# 1. Energy data

## 1.0 Startup

In [11]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


## 1.1 Load data

In [12]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'N' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
%opts.numOptFC = 25 ;
opts.numOptFC = 5 ;

## 1.2 Test

partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'PoE', 'GPoE', 'BCM', 'RBCM', 'GRBCM', 'KLRBCM', 'KLGRBCM'};

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        [models,t_dGP_train] = aggregation_train(x,y,opts) ;
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
        filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor


partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'};
qs = 1.5:0.1:1.7;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        for q = qs
            printf('Processing %s %s %s %6.2f\r\n', dataset, partitionCriterion, criterion, q);
            opts.partitionCriterion = partitionCriterion ;
            [models,t_dGP_train] = aggregation_train(x,y,opts) ;
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
            filename = sprintf('./res/%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


## 1.3 Evaluate all

In [4]:
opts.numOptFC = 25 ;

In [8]:
partitionCriterions = {'kmeans', 'random'}; %, 'kmeans', 'knkmeans'
criterions = {'PoE', 'GPoE', 'BCM', 'RBCM', 'GRBCM', 'KLRBCM', 'KLGRBCM'};

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        [models,t_dGP_train] = aggregation_train(x,y,opts) ;
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
        filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor


Processing energy_data kmeans PoE
Linesearch     25;  Value -2.767005e+04
PoE: SMSE 0.0028, MSLL -1.9060, NLPD 0.9582
Processing energy_data kmeans GPoE
Linesearch     25;  Value -2.774167e+04
GPoE: SMSE 0.0024, MSLL 10.2395, NLPD 13.1038
Processing energy_data kmeans BCM
Linesearch     25;  Value -2.745659e+04
BCM: SMSE 0.0019, MSLL -3.2271, NLPD -0.3629
Processing energy_data kmeans RBCM
Linesearch     25;  Value -2.762962e+04
RBCM: SMSE 0.0025, MSLL 15.2245, NLPD 18.0887
Processing energy_data kmeans GRBCM
Linesearch     25;  Value -2.764679e+04
GRBCM: SMSE 0.0019, MSLL -4.0041, NLPD -1.1399
Processing energy_data kmeans KLRBCM
Linesearch     25;  Value -2.750830e+04
KLRBCM: SMSE 0.0020, MSLL 10.6057, NLPD 13.4699
Processing energy_data kmeans KLGRBCM
Linesearch     25;  Value -2.764503e+04
KLGRBCM: SMSE 0.0049, MSLL 24710.9626, NLPD 24713.8269
Processing energy_data random PoE
Linesearch     25;  Value -2.524964e+04
PoE: SMSE 0.0030, MSLL 21.1199, NLPD 23.9842
Processing energy_dat

In [14]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        [models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
Linesearch      5;  Value -2.177476e+04
TERBCM: SMSE 0.0011, MSLL 0.1161, NLPD 2.9567 - q   0.20
TERBCM: SMSE 0.0010, MSLL -2.1336, NLPD 0.7070 - q   0.40
TERBCM: SMSE 0.0010, MSLL -3.0224, NLPD -0.1818 - q   0.60
TERBCM: SMSE 0.0010, MSLL -3.4367, NLPD -0.5961 - q   0.80
TERBCM: SMSE 0.0009, MSLL -3.5945, NLPD -0.7539 - q   1.00
TERBCM: SMSE 0.0009, MSLL -3.5624, NLPD -0.7218 - q   1.20
TERBCM: SMSE 0.0009, MSLL -3.3431, NLPD -0.5025 - q   1.40
TERBCM: SMSE 0.0008, MSLL -2.8949, NLPD -0.0543 - q   1.60
TERBCM: SMSE 0.0008, MSLL -2.1307, NLPD 0.7099 - q   1.80
TERBCM: SMSE 0.0008, MSLL -0.9037, NLPD 1.9369 - q   2.00
TERBCM: SMSE 0.0008, MSLL 1.0207, NLPD 3.8613 - q   2.20
TERBCM: SMSE 0.0008, MSLL 4.0120, NLPD 6.8526 - q   2.40
TERBCM: SMSE 0.0008, MSLL 8.6521, NLPD 11.4927 - q   2.60
TERBCM: SMSE 0.0008, MSLL 15.8615, NLPD 18.7021 - q   2.80
TERBCM: SMSE 0.0008, MSLL 27.1023, NLPD 29.9429 - q   3.00


## 1.4 Find q

partitionCriterion = 'kmeans'; % 'random', 'kmeans', 'knkmeans'
criterion = 'TERBCM'; %, 'TEGRBCM' 'TEGPoE'};
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;

In [15]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=0.000001, lambda=10, init_q=1.5);
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm =   -2.3343e+06
grad_q_reg = -826.64
ttb =    7.5241e+04
Optimizing q: remain iter - 49, q - 1.4995
grad_q_norm =   -7.1681e+06
grad_q_reg = -1920.9
ttb =    7.0402e+04
Optimizing q: remain iter - 48, q - 1.4981
grad_q_norm =   -1.5002e+06
grad_q_reg = -843.91
ttb =    9.0341e+04
Optimizing q: remain iter - 47, q - 1.4977
grad_q_norm =   -3.7526e+06
grad_q_reg = -1255.1
ttb =    9.5401e+04
Optimizing q: remain iter - 46, q - 1.4970
grad_q_norm =   -2.8505e+06
grad_q_reg = -1145.0
ttb =    8.9994e+04
Optimizing q: remain iter - 45, q - 1.4964
grad_q_norm =   -2.1932e+06
grad_q_reg = -1496.1
ttb =    1.2454e+05
Optimizing q: remain iter - 44, q - 1.4959
grad_q_norm =   -7.0945e+06
grad_q_reg = -164.32
ttb =    1.7712e+04
Optimizing q: remain iter - 43, q - 1.4945
grad_q_norm =   -5.4112e+06
grad_q_reg = -1893.4
ttb =    7.1181e+04
Optimizing q: remain iter - 42, q - 1.4934
grad_q_norm =   -1.1147e+06
grad_q_reg =  499.69

## 1.5 Find q, use Ynorm

In [4]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
%opts.numOptFC = 25 ;
opts.numOptFC = 5 ;

In [5]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        [models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
Linesearch      5;  Value -3.677169e+04
TERBCM: SMSE 0.0017, MSLL 0.4527, NLPD 3.2857 - q   0.20
TERBCM: SMSE 0.0017, MSLL -0.4173, NLPD 2.4157 - q   0.40
TERBCM: SMSE 0.0017, MSLL -0.1212, NLPD 2.7118 - q   0.60
TERBCM: SMSE 0.0017, MSLL 1.1140, NLPD 3.9470 - q   0.80
TERBCM: SMSE 0.0017, MSLL 3.9071, NLPD 6.7402 - q   1.00
TERBCM: SMSE 0.0017, MSLL 9.9726, NLPD 12.8056 - q   1.20
TERBCM: SMSE 0.0018, MSLL 23.6018, NLPD 26.4348 - q   1.40
TERBCM: SMSE 0.0018, MSLL 55.7901, NLPD 58.6232 - q   1.60
TERBCM: SMSE 0.0018, MSLL 135.7341, NLPD 138.5671 - q   1.80
TERBCM: SMSE 0.0018, MSLL 343.6701, NLPD 346.5031 - q   2.00
TERBCM: SMSE 0.0019, MSLL 907.0512, NLPD 909.8842 - q   2.20
TERBCM: SMSE 0.0019, MSLL 2488.7742, NLPD 2491.6072 - q   2.40
TERBCM: SMSE 0.0019, MSLL 7069.2577, NLPD 7072.0908 - q   2.60
TERBCM: SMSE 0.0019, MSLL 20697.9981, NLPD 20700.8311 - q   2.80
TERBCM: SMSE 0.0019, MSLL 62227.6163, NLPD 62230.4493 - q   3.00


In [6]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=150, bs=100, lr=0.000001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 150, q - 1.5000
grad_q_norm =   -3.5801e+06
grad_q_reg = -4320.0
ttb =    3.4137e+05
Optimizing q: remain iter - 149, q - 1.4993
grad_q_norm =   -2.7674e+07
grad_q_reg = -2742.7
ttb =    4.1480e+05
Optimizing q: remain iter - 148, q - 1.4936
grad_q_norm =   -2.3820e+06
grad_q_reg = -2665.4
ttb =    3.1562e+05
Optimizing q: remain iter - 147, q - 1.4931
grad_q_norm =   -1.6891e+06
grad_q_reg = -706.88
ttb =    2.5250e+05
Optimizing q: remain iter - 146, q - 1.4928
grad_q_norm =   -1.3952e+08
grad_q_reg =   -2.5316e+04
ttb =    4.5494e+05
Optimizing q: remain iter - 145, q - 1.4642
grad_q_norm =   -4.4314e+07
grad_q_reg =   -2.2818e+04
ttb =    4.8551e+05
Optimizing q: remain iter - 144, q - 1.4552
grad_q_norm =   -5.8819e+06
grad_q_reg = -4051.9
ttb =    3.7197e+05
Optimizing q: remain iter - 143, q - 1.4539
grad_q_norm =   -7.9499e+06
grad_q_reg = -9257.2
ttb =    3.4916e+05
Optimizing q: remain iter - 142, q - 1.4523
grad_q_norm =   -1.4554e+07

In [8]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q=0.4929) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

## 1.6 Multi find q

In [13]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
%opts.numOptFC = 25 ;
opts.numOptFC = 5 ;

In [14]:
partitionCriterion = 'kmeans'; % 'random', 'kmeans', 'knkmeans'
criterion = 'TERBCM'; %, 'TEGRBCM' 'TEGPoE'};
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;

Linesearch      5;  Value -3.691092e+04


In [17]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.01, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.0466
   1.7006
   1.9208
   1.7440
   1.4074
   1.9218
   1.4065
   1.7490
   1.0116
   1.7248
   2.0009
   1.9801
   1.2431
   1.7415
   1.9896
   1.0599
   1.8669
   1.9217
   1.3078
   1.9860
   1.6127
   1.6707
   1.9978
   1.3025
   1.7051
   1.3390
   1.3421
   1.3764
   1.4472
   1.6075
   1.5191
   1.6234
   1.7606
   1.6430
   1.9123
   1.2984
   1.0906
   1.2718
   1.5285
   1.9403
   1.9977
   1.0815
   1.5997
   1.5868
   1.0749
   1.2275
   1.2264
   1.0201
   1.5909
   1.8428

ttb =    1.3460e+07
Optimizing qs: remain iter - 49
qs =

   1.04327
   1.70052
   1.92077
   1.74408
   1.40736
   1.92182
   1.40657
   1.74902
   1.01134
   1.72498
   1.97632
   1.98001
   1.24310
   1.74138
   1.98891
   1.05969
   1.86689
   1.92169
   1.30775
   1.98594
   1.61248
   1.67063
   1.99759
   1.30218
   1.70261
   1.33960
   1.34201
   1.37624
   1.44720
   1.60690
   1.51850
   1.62343
   1.76036
   1.63547
   1.11234
   1.

  -3.2463
  -3.8971
  -6.5456
  -5.1736
  -5.3799
  -4.8099
  -5.3541

ttb = -2.0040e+04 - 2.4542e-12i
Optimizing qs: remain iter - 37
qs =

  -6.1761
  -5.5303
  -5.2794
  -4.6559
  -5.7932
  -5.2811
  -5.0100
  -6.2511
  -6.2076
  -5.4755
  -5.2236
  -3.6204
  -5.9569
  -3.9183
  -5.2143
  -6.1401
  -2.1332
  -3.6785
  -5.8935
  -4.8252
  -3.9876
  -5.5297
  -6.8025
  -5.9381
  -5.4982
  -5.8588
  -5.8581
  -5.9196
  -5.7528
  -5.6138
  -2.4823
  -4.7766
  -5.4396
  -2.3659
  -6.0894
  -5.9033
  -2.9145
  -6.7607
  -5.6743
  -5.2780
  -5.0167
  -6.9175
  -7.4085
  -4.0463
  -4.6971
  -7.3456
  -5.9736
  -6.1799
  -5.6099
  -6.1541

ttb = -6.0165e+04 - 7.3680e-12i
Optimizing qs: remain iter - 36
qs =

  -6.9761
  -6.3303
  -6.0794
  -5.4559
  -6.5932
  -6.0811
  -5.8100
  -7.0511
  -7.0076
  -6.2755
  -6.0236
  -4.4204
  -6.7569
  -4.7183
  -6.0143
  -6.9401
  -2.9332
  -4.4785
  -6.6935
  -5.6252
  -4.7876
  -6.3297
  -7.6025
  -6.7381
  -6.2982
  -6.6588
  -6.6581
  -6.7196
  -6.552

  -16.320
  -14.553
  -14.414
  -12.882
  -15.177
  -14.240
  -12.766
  -14.889
  -16.303
  -10.115
  -17.161
  -16.074
  -15.678
  -15.417
  -12.517
  -16.209
  -14.446
  -15.097
  -16.146
  -16.374
  -14.980
  -16.010
  -14.954

ttb = -4.2057e+06 - 5.1505e-10i
Optimizing qs: remain iter - 23
qs =

  -17.376
  -16.730
  -16.479
  -14.256
  -16.993
  -14.881
  -14.610
  -15.851
  -17.408
  -15.075
  -14.824
  -13.220
  -15.557
  -13.518
  -16.414
  -15.740
  -13.333
  -13.278
  -15.494
  -16.025
  -15.188
  -15.130
  -16.402
  -17.138
  -16.698
  -15.459
  -17.058
  -17.120
  -15.353
  -15.214
  -13.682
  -15.977
  -15.040
  -13.566
  -14.089
  -17.103
  -10.915
  -17.961
  -16.874
  -16.478
  -16.217
  -13.317
  -17.009
  -15.246
  -15.897
  -16.946
  -17.174
  -15.780
  -16.810
  -15.754

ttb = -7.7845e+06 - 9.5332e-10i
Optimizing qs: remain iter - 22
qs =

  -18.176
  -17.530
  -17.279
  -15.056
  -17.793
  -15.681
  -13.810
  -16.651
  -18.208
  -15.875
  -15.624
  -14.020
  -16.35

  -25.974
  -24.580
  -20.810
  -21.354

ttb = -2.9238e+08 - 3.5806e-08i
Optimizing qs: remain iter - 9
qs =

  -20.576
  -21.530
  -24.479
  -20.656
  -21.793
  -21.281
  -21.010
  -22.251
  -20.608
  -21.475
  -21.224
  -22.820
  -25.157
  -21.518
  -21.214
  -22.140
  -22.933
  -22.878
  -21.894
  -20.825
  -23.188
  -21.530
  -21.202
  -21.938
  -21.498
  -21.859
  -23.458
  -21.920
  -21.753
  -21.614
  -21.682
  -20.777
  -21.440
  -21.566
  -22.089
  -21.903
  -22.115
  -22.761
  -21.674
  -24.478
  -19.417
  -22.917
  -21.809
  -21.646
  -20.697
  -20.146
  -25.174
  -25.380
  -20.010
  -20.554

ttb = -2.5242e+08 - 3.0912e-08i
Optimizing qs: remain iter - 8
qs =

  -21.376
  -22.330
  -25.279
  -21.456
  -22.593
  -22.081
  -21.810
  -23.051
  -21.408
  -20.675
  -22.024
  -22.020
  -25.957
  -22.318
  -22.014
  -21.340
  -23.733
  -22.078
  -21.094
  -20.025
  -22.388
  -22.330
  -22.002
  -21.138
  -20.698
  -21.059
  -24.258
  -21.120
  -22.553
  -22.414
  -20.882
  -21.577


In [19]:
qs
save('./res/energydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

  -21.376
  -20.730
  -26.879
  -21.456
  -20.993
  -22.081
  -20.210
  -21.451
  -21.408
  -20.675
  -20.424
  -22.020
  -29.157
  -20.718
  -22.014
  -22.940
  -23.733
  -22.078
  -21.094
  -20.025
  -23.988
  -20.730
  -20.402
  -21.138
  -20.698
  -21.059
  -22.658
  -21.120
  -20.953
  -20.814
  -22.482
  -19.977
  -22.240
  -20.766
  -21.289
  -21.103
  -21.315
  -20.361
  -20.874
  -22.078
  -20.217
  -22.117
  -21.009
  -20.846
  -21.497
  -19.346
  -25.974
  -30.980
  -20.810
  -21.354

TERBCM: SMSE 0.0033, MSLL -7.3694, NLPD -4.5282


In [20]:
save('./res/energydataqs_res', 'SMSE','MSLL','NLPD')

## 1.7 Multi find q, big lambda

In [4]:
partitionCriterion = 'kmeans'; % 'random', 'kmeans', 'knkmeans'
criterion = 'TERBCM'; %, 'TEGRBCM' 'TEGPoE'};
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;

Linesearch      5;  Value -2.206245e+04


In [16]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.01, lambda=10000, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
qs
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.1864
   1.0285
   1.8689
   1.1847
   1.7811
   1.7949
   1.9895
   1.5273
   1.1121
   1.1134
   1.7239
   1.0820
   2.0038
   1.3162
   1.5980
   1.3113
   1.7353
   1.1611
   1.8527
   1.0862
   1.5770
   1.0463
   1.3661
   1.0769
   1.8021
   1.0185
   1.6552
   1.4053
   1.7619
   1.6657
   1.7114
   1.9906
   1.9987
   1.8730
   1.0172
   1.7217
   1.5754
   1.5880
   1.5198
   1.2672
   1.5195
   1.0242
   1.9773
   1.8819
   1.0108
   1.3856
   1.2478
   1.8897
   1.5225
   1.9574

ttb =    3.9837e+05
Optimizing qs: remain iter - 49
qs =

   0.38644
   1.04270
   1.90943
   1.19027
   1.78739
   1.79487
   1.98966
   1.52729
   1.29971
   1.11375
   0.92386
   1.08204
   2.03024
   0.51625
   1.59830
   1.31130
   0.93527
   1.16449
   1.85281
   1.10449
   1.57701
   1.14630
   0.56606
   1.09301
   1.80223
   1.01963
   1.65201
   1.42394
   0.96190
   1.73960
   1.71142
   1.99076
   1.99868
   1.87319
   1.01792
   0.

   1.38306
   1.31122
   1.03897
   1.37207
   1.49620
   1.62646
   1.41400
   0.96337
   1.58355
   1.18150
   0.91307
   0.91885
   1.76271
   1.35729
   0.62349
   1.22290
   1.15315
   1.51611
   0.89312
   0.82248
   1.03477
   1.35827
   1.12588
   1.07885
   1.41668
   1.81077
   1.38362
   0.81543
   1.42777
   0.82073
   1.39017
   1.19889
   1.31380
   0.92011
   1.39553

ttb =    1.6021e+05
Optimizing qs: remain iter - 36
qs =

   0.421568
   1.595511
   0.556991
   1.481196
   1.407789
   1.185214
   1.066504
   1.388949
   1.341144
   1.196314
   0.712734
   1.014879
   0.442124
   0.768988
   1.234068
   1.385952
   1.379481
   1.203494
   1.437350
   1.638995
   1.627607
   1.565075
   0.163368
   1.514176
   1.235420
   1.175795
   1.109274
   1.796206
   1.482770
   0.929313
   1.230742
   1.272385
   1.521447
   1.090517
   1.076067
   1.054660
   1.427537
   1.138753
   1.220687
   1.428406
   1.010769
   1.427995
   1.328952
   1.004416
   0.020726
   1.331655
   1

   1.33704
   1.34068
   0.80339
   1.40069
   1.16734
   1.57075
   0.71902
   1.15686
   1.39406
   1.20064
   1.29329
   1.02996
   1.64690
   1.18462
   1.25760
   1.39965
   0.37887
   1.20202
   1.28135
   1.45501
   1.51340
   0.45673
   0.59612
   0.66088
   0.46747
   1.19882
   1.72123
   1.41619
   0.74957

ttb =    2.0970e+05
Optimizing qs: remain iter - 23
qs =

   0.70595
   0.77639
   1.12556
  -0.14435
   1.65297
   1.21804
   1.23630
   1.20587
   1.32277
   0.72543
   1.49862
   1.49564
   1.36517
   0.92690
   1.42732
   1.59713
   1.00939
   0.88585
   0.60200
   1.15422
   1.69683
   1.39296
   1.35595
   1.29946
   1.44879
   0.36734
   1.57089
   1.51902
   1.15834
   1.40395
   1.23876
   1.29329
   1.16718
   1.73042
   1.28193
   1.25793
   0.81214
   1.17887
   1.25232
   1.28166
   1.38669
   1.51895
   1.25673
   0.68863
   1.46088
   1.26747
   1.19883
   1.72123
   1.41801
   1.54957

ttb =  6.9809e+05 - 3.8657e+03i
Optimizing qs: remain iter - 22
qs =

 

   0.354219
  -3.903169
  -5.807043
  -2.644051
   3.699459
  -7.351206
  -6.832655
   0.770888
   0.719021
  -6.041658
   0.603951
   0.438756
   2.093289
  -6.032820
   0.930418
  -5.918071
  -7.542069
   4.812142
  -6.021134
  -7.547679
  -7.518339
   0.586692
   0.718953
  -5.943269
  -0.111366
   0.660881
  -5.932528
  -7.601168
  -7.078766
  -5.781990
  -7.250426

ttb =  2.7742e+10 - 3.0834e+06i
Optimizing qs: remain iter - 11
qs =

   8.7059546
   0.7763919
  -6.8744407
  -9.7443465
   0.0529733
  -6.7819646
  -6.7636975
  -6.7941266
  -0.2772294
   0.7254286
  -0.1013761
  -0.1043580
  -8.2348329
  -0.6731042
   1.4273215
  -0.0028714
  -6.9906081
   0.8858474
  -8.9980041
  -0.4457815
  -4.7031688
  -6.6070435
  -3.4440510
   2.8994589
  -8.1512061
  -7.6326551
  -0.0291115
  -0.0809792
  -6.8416585
  -0.1960490
   1.2387556
   1.2932893
  -6.8328196
   0.1304184
  -6.7180706
  -8.3420692
   4.0121425
  -6.8211342
  -8.3476788
  -8.3183386
  -0.2133081
   1.5189525
  -6.743268

  -1.1642e+01
  -8.1960e+00
  -5.1612e+00
   2.8933e+00
  -1.0033e+01
  -1.4696e+00
  -9.9181e+00
  -1.1542e+01
   8.1214e-01
  -1.1621e+01
  -9.9477e+00
  -1.3118e+01
   1.3867e+00
  -1.6810e+00
  -8.3433e+00
  -2.5114e+00
   1.4609e+00
  -9.9325e+00
  -1.3201e+01
  -1.4279e+01
  -9.7820e+00
  -9.6504e+00

ttb =  7.7497e+47 - 6.7926e+25i
qs =

   17.505955
    1.576392
  -12.474441
   -8.944347
    0.852973
   -9.181965
  -10.763697
  -10.794127
   -9.077229
   -0.074571
    0.698624
   -4.104358
  -13.834833
   -9.473104
   -2.572678
    0.797129
  -12.590608
   -7.914153
   -9.798004
    1.954219
  -13.503169
   -9.007043
   -9.044051
   -1.100541
   -8.951206
   -8.432655
   -8.829112
    0.719021
  -10.841658
   -8.996049
   -5.961244
    3.693289
   -9.232820
   -2.269582
   -9.118071
  -10.742069
    0.012142
  -10.821134
   -9.147679
  -12.318339
    2.186692
   -2.481047
   -9.143269
   -3.311366
    2.260881
   -9.132528
  -14.001168
  -15.078766
  -10.581990
   -8.850426

TE

# 2. Protein

## 2.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


## 2.1 Load data

In [21]:
dataset = 'protein'
load ('./data/protein.mat');
[ndata, ~] = size(M);
X = M(:,2:9);
Y = M(:,1);
R = randperm(ndata);        
xt = X(R(1:5730),:);
yt = Y(R(1:5730));
R(1:5730) = [];
x = X(R,:);          
y = Y(R);
n = 40000; nt = 5730;   %Ms = 50

dataset = protein


## 2.2 Evaluate all

In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
%partitionCriterion = 'kmeans' ; % 'random', 'kmeans', 'knkmeans'
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'N' ;
opts.Ms = 50 ; 
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
opts.numOptFC = 10 ;

In [7]:
opts.numOptFC = 10 ;

partitionCriterions = {'kmeans', 'random'}; %, 'kmeans', 'knkmeans'
criterions = {'PoE', 'GPoE', 'BCM', 'RBCM', 'GRBCM', 'KLRBCM', 'KLGRBCM'};

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        [models,t_dGP_train] = aggregation_train(x,y,opts) ;
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
        filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor


In [8]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        [models,t_dGP_train] = aggregation_train(x,y,opts) ;
        save('./res/model', 'models','t_dGP_train');
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing protein kmeans TERBCM





## 2.3 Find q

opts.numOptFC = 5 ;
partitionCriterion = 'kmeans';
criterion = 'TERBCM';
printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;

printf('Optimizing q\r\n');
q = grad_q_all(models, criterion, opts, iter=200, bs=100, lam=0.000000000001, init_q=1.5);
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

In [4]:
s = load('./res/model');
models = s.models;
t_dGP_train = s.t_dGP_train;
partitionCriterion = 'kmeans'; % 'random', 'kmeans', 'knkmeans'
criterion = 'TERBCM'; %, 'TEGRBCM' 'TEGPoE'};

In [9]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=500, lr=1.0, lambda=0.01, init_q=1.5);
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
error: 'models' undefined near line 1 column 12
error: evaluating argument list element number 1
error: 'models' undefined near line 1 column 55
error: evaluating argument list element number 2
error: 'mu_dGP' undefined near line 1 column 42
error: evaluating argument list element number 5
error: 'SMSE' undefined near line 1 column 65
error: evaluating argument list element number 3


## 2.4 Find q, use Ynorm

In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
%opts.numOptFC = 25 ;
opts.numOptFC = 5 ;

In [4]:
opts.numOptFC = 5 ;
partitionCriterion = 'kmeans';
criterion = 'TERBCM';
printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;

Processing protein kmeans TERBCM
Linesearch      5;  Value 3.757011e+04


In [6]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        save('./res/model', 'models','t_dGP_train');
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing protein kmeans TERBCM
TERBCM: SMSE 14.4709, MSLL 0.8521, NLPD 4.0787 - q   0.20
TERBCM: SMSE 14.1857, MSLL 0.0934, NLPD 3.3200 - q   0.40
TERBCM: SMSE 14.2295, MSLL -0.2031, NLPD 3.0235 - q   0.60
TERBCM: SMSE 14.4424, MSLL -0.3497, NLPD 2.8769 - q   0.80
TERBCM: SMSE 14.7754, MSLL -0.4262, NLPD 2.8004 - q   1.00
TERBCM: SMSE 15.2056, MSLL -0.4638, NLPD 2.7628 - q   1.20
TERBCM: SMSE 15.7183, MSLL -0.4781, NLPD 2.7485 - q   1.40
TERBCM: SMSE 16.3018, MSLL -0.4776, NLPD 2.7490 - q   1.60
TERBCM: SMSE 16.9456, MSLL -0.4676, NLPD 2.7590 - q   1.80
TERBCM: SMSE 17.6395, MSLL -0.4516, NLPD 2.7750 - q   2.00
TERBCM: SMSE 18.3733, MSLL -0.4317, NLPD 2.7949 - q   2.20
TERBCM: SMSE 19.1370, MSLL -0.4096, NLPD 2.8170 - q   2.40
TERBCM: SMSE 19.9210, MSLL -0.3864, NLPD 2.8402 - q   2.60
TERBCM: SMSE 20.7162, MSLL -0.3629, NLPD 2.8637 - q   2.80
TERBCM: SMSE 21.5142, MSLL -0.3397, NLPD 2.8869 - q   3.00


In [5]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=500, lr=1.0, lambda=0.01, init_q=1.5);
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm = 0
grad_q_reg = 0
ttb =  0.39141
Optimizing q: remain iter - 49, q - 1.5000
grad_q_norm =    4.8519e+04
grad_q_reg =  38.894
ttb =  398.75
Optimizing q: remain iter - 48, q - 2.3000
grad_q_norm =    6.4176e+04
grad_q_reg =  22.482
ttb =  299.25
Optimizing q: remain iter - 47, q - 3.1000
grad_q_norm =    1.9118e+04
grad_q_reg =  4.0811
ttb =  62.309
Optimizing q: remain iter - 46, q - 3.8803
grad_q_norm =  3343.1
grad_q_reg =  0.34168
ttb =  8.2912
Optimizing q: remain iter - 45, q - 4.0168
grad_q_norm =  6377.6
grad_q_reg =  1.3069
ttb =  20.298
Optimizing q: remain iter - 44, q - 4.2771
grad_q_norm =  1043.5
grad_q_reg =  0.14763
ttb =  4.5939
Optimizing q: remain iter - 43, q - 4.3263
grad_q_norm =  1601.6
grad_q_reg =  0.13385
ttb =  4.3584
Optimizing q: remain iter - 42, q - 4.3916
grad_q_norm =  5887.3
grad_q_reg =  0.71431
ttb =  12.450
Optimizing q: remain iter - 41, q - 4.6319
grad_q_norm =  737.80
grad_q_reg 

## 2.5 Multi find q

In [22]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
%opts.numOptFC = 25 ;
opts.numOptFC = 5 ;

In [23]:
partitionCriterion = 'kmeans'; % 'random', 'kmeans', 'knkmeans'
criterion = 'TERBCM'; %, 'TEGRBCM' 'TEGPoE'};
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;

Linesearch      5;  Value 3.791484e+04


In [24]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.01, lambda=10, init_q=iqs);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.6328
   1.3212
   1.5795
   1.3996
   2.0033
   1.5551
   1.6494
   1.7034
   1.2340
   1.9084
   1.7327
   1.4112
   1.0990
   1.3643
   1.5852
   1.7728
   1.6859
   1.1987
   1.2556
   1.3310
   1.5975
   1.6065
   1.9502
   1.8296
   1.4895
   1.7385
   1.6318
   1.3084
   1.1335
   1.2022
   1.0714
   1.0227
   1.2148
   1.4317
   1.0916
   1.1705
   1.7447
   1.5940
   1.8786
   1.1343
   1.6125
   1.7128
   1.5230
   2.0000
   1.7001
   1.8471
   1.1334
   1.6207
   1.5252
   1.4458

ttb =  274.78
Optimizing qs: remain iter - 49
qs =

   1.6328
   1.3213
   1.5812
   1.3997
   2.0035
   1.5551
   1.6498
   1.7034
   1.2340
   1.9088
   1.7332
   1.4121
   1.0990
   1.3643
   1.5854
   1.7728
   1.6859
   1.1987
   1.2573
   1.3311
   1.5977
   1.6067
   1.9512
   1.8298
   1.4895
   1.7386
   1.6318
   1.3098
   1.1337
   1.2050
   1.0714
   1.0230
   1.2151
   1.4338
   1.0916
   1.1708
   1.7447
   1.5944
   1.8786
   1.1

   1.1395
   1.6144
   1.7180
   1.5264
   2.0000
   1.7007
   1.8478
   1.1349
   1.6207
   1.5262
   1.4482

ttb =  340.22
Optimizing qs: remain iter - 35
qs =

   1.6403
   1.3222
   1.5906
   1.3998
   2.0052
   1.5553
   1.6537
   1.7034
   1.2347
   1.9096
   1.7348
   1.4131
   1.0990
   1.3643
   1.5855
   1.7732
   1.6859
   1.1987
   1.2639
   1.3323
   1.5984
   1.6080
   1.9515
   1.8300
   1.4898
   1.7386
   1.6321
   1.3143
   1.1347
   1.2119
   1.0714
   1.0247
   1.2162
   1.4387
   1.0916
   1.1718
   1.7449
   1.5953
   1.8792
   1.1416
   1.6144
   1.7182
   1.5264
   2.0000
   1.7007
   1.8478
   1.1349
   1.6207
   1.5262
   1.4482

ttb =  19.481
Optimizing qs: remain iter - 34
qs =

   1.6404
   1.3222
   1.5906
   1.3998
   2.0052
   1.5553
   1.6537
   1.7034
   1.2347
   1.9096
   1.7348
   1.4131
   1.0990
   1.3643
   1.5855
   1.7732
   1.6859
   1.1987
   1.2639
   1.3326
   1.5984
   1.6080
   1.9515
   1.8300
   1.4898
   1.7386
   1.6321
   1.3143
   1

   1.1349
   1.2191
   1.0714
   1.0263
   1.2171
   1.4447
   1.0916
   1.1765
   1.7449
   1.5961
   1.8794
   1.1492
   1.6156
   1.7229
   1.5281
   2.0000
   1.7011
   1.8481
   1.1360
   1.6207
   1.5270
   1.4529

ttb =  2.2936
Optimizing qs: remain iter - 20
qs =

   1.6499
   1.3222
   1.5963
   1.3999
   2.0061
   1.5553
   1.6547
   1.7034
   1.2366
   1.9129
   1.7357
   1.4206
   1.0990
   1.3643
   1.5856
   1.7732
   1.6859
   1.1987
   1.2701
   1.3330
   1.5989
   1.6105
   1.9535
   1.8301
   1.4928
   1.7386
   1.6321
   1.3199
   1.1349
   1.2191
   1.0714
   1.0263
   1.2171
   1.4447
   1.0916
   1.1765
   1.7449
   1.5961
   1.8794
   1.1492
   1.6156
   1.7229
   1.5281
   2.0000
   1.7011
   1.8481
   1.1360
   1.6207
   1.5270
   1.4529

ttb =  56.713
Optimizing qs: remain iter - 19
qs =

   1.6502
   1.3232
   1.5963
   1.3999
   2.0061
   1.5553
   1.6547
   1.7034
   1.2366
   1.9129
   1.7357
   1.4206
   1.0990
   1.3643
   1.5860
   1.7732
   1.6859
   1

   1.2759
   1.3330
   1.5990
   1.6110
   1.9540
   1.8307
   1.4928
   1.7386
   1.6323
   1.3199
   1.1349
   1.2238
   1.0714
   1.0261
   1.2176
   1.4491
   1.0916
   1.1765
   1.7452
   1.5969
   1.8798
   1.1501
   1.6188
   1.7236
   1.5312
   2.0000
   1.7012
   1.8481
   1.1366
   1.6207
   1.5274
   1.4559

ttb =  20.622
Optimizing qs: remain iter - 5
qs =

   1.6547
   1.3250
   1.6024
   1.3999
   2.0061
   1.5555
   1.6563
   1.7034
   1.2371
   1.9139
   1.7376
   1.4206
   1.0990
   1.3643
   1.5867
   1.7733
   1.6859
   1.1987
   1.2759
   1.3332
   1.5990
   1.6110
   1.9540
   1.8307
   1.4928
   1.7386
   1.6323
   1.3199
   1.1349
   1.2238
   1.0714
   1.0261
   1.2179
   1.4491
   1.0916
   1.1765
   1.7452
   1.5969
   1.8799
   1.1501
   1.6188
   1.7236
   1.5312
   2.0000
   1.7012
   1.8481
   1.1366
   1.6207
   1.5276
   1.4559

ttb =  489.61
Optimizing qs: remain iter - 4
qs =

   1.6562
   1.3250
   1.6047
   1.3999
   2.0061
   1.5555
   1.6563
   1.7

In [25]:
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
qs
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.6572
   1.3250
   1.6061
   1.3999
   2.0061
   1.5555
   1.6588
   1.7034
   1.2371
   1.9144
   1.7385
   1.4227
   1.0990
   1.3643
   1.5867
   1.7733
   1.6859
   1.1987
   1.2810
   1.3332
   1.5990
   1.6110
   1.9542
   1.8307
   1.4928
   1.7386
   1.6323
   1.3207
   1.1349
   1.2286
   1.0714
   1.0266
   1.2179
   1.4494
   1.0916
   1.1765
   1.7452
   1.5973
   1.8799
   1.1517
   1.6188
   1.7244
   1.5316
   2.0000
   1.7012
   1.8483
   1.1366
   1.6207
   1.5276
   1.4564

TERBCM: SMSE 16.2053, MSLL -0.4855, NLPD 2.7453


In [26]:
save('./res/proteinqs', 'qs');
save('./res/protein_res', 'SMSE','MSLL','NLPD')

# 3. Video

In [5]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [6]:
dataset = 'video'
load ('./data/video_data.mat');
[ndata, ~] = size(video_data);
X = video_data(:,[1:7,9:11,13:18]);
Y = video_data(:,19);
R = randperm(ndata);        
xt = X(R(1:18784),:);
yt = Y(R(1:18784));
R(1:18784) = [];
x = X(R,:);          
y = Y(R);
% x = x1(1:45000,:);
% y = y1(1:45000);
% xs = x1(45001:50000,:);
% ys = y1(45001:50000);
n = 50000; nt = 18784;   %Ms = 80

dataset = video


## 3.1 Find q, use Ynorm

In [7]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;
%opts.numOptFC = 25 ;
opts.numOptFC = 5 ;

In [8]:
opts.numOptFC = 5 ;
partitionCriterion = 'kmeans';
criterion = 'TERBCM';
printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;

Processing video kmeans TERBCM
Linesearch      5;  Value 2.595853e+04


In [9]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        save('./res/model', 'models','t_dGP_train');
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing video kmeans TERBCM
TERBCM: SMSE 39.7844, MSLL 5.9104, NLPD 10.1288 - q   0.20
TERBCM: SMSE 39.5064, MSLL 2.5046, NLPD 6.7229 - q   0.40
TERBCM: SMSE 39.2780, MSLL 1.0393, NLPD 5.2577 - q   0.60
TERBCM: SMSE 39.1001, MSLL 0.2335, NLPD 4.4519 - q   0.80
TERBCM: SMSE 38.9782, MSLL -0.2542, NLPD 3.9641 - q   1.00
TERBCM: SMSE 38.9167, MSLL -0.5626, NLPD 3.6558 - q   1.20
TERBCM: SMSE 38.9194, MSLL -0.7607, NLPD 3.4577 - q   1.40
TERBCM: SMSE 38.9908, MSLL -0.8874, NLPD 3.3310 - q   1.60
TERBCM: SMSE 39.1377, MSLL -0.9663, NLPD 3.2521 - q   1.80
TERBCM: SMSE 39.3691, MSLL -1.0127, NLPD 3.2057 - q   2.00
TERBCM: SMSE 39.6963, MSLL -1.0365, NLPD 3.1819 - q   2.20
TERBCM: SMSE 40.1317, MSLL -1.0447, NLPD 3.1737 - q   2.40
TERBCM: SMSE 40.6878, MSLL -1.0420, NLPD 3.1764 - q   2.60
TERBCM: SMSE 41.3762, MSLL -1.0318, NLPD 3.1866 - q   2.80
TERBCM: SMSE 42.2058, MSLL -1.0165, NLPD 3.2019 - q   3.00


In [10]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=500, lr=1.0, lambda=0.01, init_q=1.5);
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm =    5.6525e+04
grad_q_reg =  1310.1
ttb =    1.5018e+04
Optimizing q: remain iter - 49, q - 2.3000
grad_q_norm =    2.2795e+05
grad_q_reg =  43.119
ttb =  335.48
Optimizing q: remain iter - 48, q - 3.1000
grad_q_norm =    2.1773e+05
grad_q_reg =  50.182
ttb =  676.38
Optimizing q: remain iter - 47, q - 3.9000
grad_q_norm =  4773.5
grad_q_reg =  15.890
ttb =  276.46
Optimizing q: remain iter - 46, q - 4.0948
grad_q_norm =    7.2010e+04
grad_q_reg =  10.447
ttb =  204.58
Optimizing q: remain iter - 45, q - 4.8948
grad_q_norm =  5998.7
grad_q_reg =  9.7583
ttb =  251.42
Optimizing q: remain iter - 44, q - 5.1397
grad_q_norm =    1.3614e+04
grad_q_reg =  3.8711
ttb =  91.018
Optimizing q: remain iter - 43, q - 5.6954
grad_q_norm =    2.2962e+04
grad_q_reg =  1.4753
ttb =  36.630
Optimizing q: remain iter - 42, q - 6.4954
grad_q_norm =  635.01
grad_q_reg =  1.4296
ttb =  45.204
Optimizing q: remain iter - 41, q - 6.5213
gr

## 3.2 Multi find q

In [11]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.01, lambda=10, init_q=iqs);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.7385
   1.9155
   1.6209
   1.3928
   1.1784
   1.9103
   1.7979
   1.7611
   1.3474
   1.2543
   1.1795
   2.0030
   1.5833
   1.3388
   1.4644
   1.0455
   1.9051
   1.6280
   1.4002
   1.2729
   1.3107
   1.3717
   1.2578
   1.7157
   1.8182
   1.7068
   1.7152
   1.7649
   1.9734
   1.2809
   1.2258
   1.1956
   1.7072
   1.9202
   1.7115
   1.2390
   1.7969
   1.2463
   1.9389
   1.3949
   1.0677
   1.8503
   1.9476
   1.9398
   1.8733
   1.8477
   1.5726
   1.2684
   1.4932
   1.1994

ttb =  1920.1
Optimizing qs: remain iter - 49
qs =

   1.7393
   1.9155
   1.6209
   1.3928
   1.1784
   1.9103
   1.7979
   1.7611
   1.3482
   1.2543
   1.1795
   2.0030
   1.5833
   1.3388
   1.4644
   1.0476
   1.9051
   1.6280
   1.4004
   1.2729
   1.3107
   1.3717
   1.2586
   1.7157
   1.8183
   1.7068
   1.7152
   1.7649
   1.9734
   1.2810
   1.2258
   1.1956
   1.7072
   1.9202
   1.7132
   1.2390
   1.7969
   1.2463
   1.9389
   1.3

   1.4017
   1.0860
   1.8503
   1.9499
   1.9398
   1.8787
   1.8483
   1.5874
   1.2837
   1.4976
   1.2123

ttb =  2691.0
Optimizing qs: remain iter - 35
qs =

   1.8812
   1.9161
   1.6218
   1.3928
   1.1821
   1.9122
   1.8048
   1.7944
   1.3500
   1.2779
   1.1944
   2.0052
   1.5897
   1.3400
   1.4840
   1.0640
   1.9057
   1.6403
   1.4144
   1.2805
   1.3107
   1.3717
   1.2712
   1.7158
   1.8291
   1.7097
   1.7318
   1.7649
   1.9737
   1.3084
   1.2563
   1.2140
   1.7087
   1.9207
   1.7173
   1.2391
   1.8276
   1.2480
   1.9403
   1.4056
   1.0926
   1.8509
   1.9545
   1.9422
   1.8843
   1.8492
   1.5878
   1.2928
   1.5028
   1.2156

ttb =  2686.7
Optimizing qs: remain iter - 34
qs =

   1.8824
   1.9161
   1.6218
   1.3928
   1.1821
   1.9122
   1.8049
   1.7945
   1.3500
   1.2779
   1.1949
   2.0052
   1.5899
   1.3400
   1.4840
   1.0640
   1.9057
   1.6404
   1.4146
   1.2805
   1.3107
   1.3717
   1.2712
   1.7158
   1.8291
   1.7097
   1.7333
   1.7649
   1

   1.9737
   1.3100
   1.2603
   1.2194
   1.7114
   1.9207
   1.7259
   1.2302
   1.8285
   1.2495
   1.9403
   1.4076
   1.0934
   1.8509
   1.9545
   1.9453
   1.8847
   1.8496
   1.5948
   1.2980
   1.5031
   1.2179

ttb =  5899.3
Optimizing qs: remain iter - 20
qs =

   1.9498
   1.9170
   1.6218
   1.3928
   1.1821
   1.9127
   1.8051
   1.7970
   1.3500
   1.2814
   1.1959
   2.0063
   1.5910
   1.3400
   1.4869
   1.0670
   1.9057
   1.6441
   1.4163
   1.2805
   1.3107
   1.3717
   1.2722
   1.7161
   1.8300
   1.7097
   1.7333
   1.7757
   1.9737
   1.3104
   1.2613
   1.2242
   1.7114
   1.9207
   1.7259
   1.2302
   1.8289
   1.2495
   1.9403
   1.4081
   1.0938
   1.8509
   1.9545
   1.9453
   1.8853
   1.8496
   1.5948
   1.3017
   1.5032
   1.2180

ttb =  3017.2
Optimizing qs: remain iter - 19
qs =

   1.9521
   1.9170
   1.6218
   1.3928
   1.1821
   1.9132
   1.8053
   1.7971
   1.3500
   1.2913
   1.2200
   2.0063
   1.5929
   1.3400
   1.4869
   1.0670
   1.9057
   1

   1.4216
   1.2962
   1.3107
   1.3717
   1.2942
   1.7161
   1.8342
   1.7152
   1.7415
   1.7757
   1.9737
   1.3170
   1.2893
   1.2298
   1.7114
   1.9212
   1.7289
   1.2250
   1.8447
   1.2522
   1.9403
   1.4134
   1.1075
   1.8523
   1.9647
   1.9493
   1.8889
   1.8520
   1.5948
   1.3092
   1.5094
   1.2208

ttb =  3704.9
Optimizing qs: remain iter - 5
qs =

   2.0334
   1.9170
   1.6218
   1.3928
   1.2046
   1.9137
   1.8088
   1.8132
   1.3535
   1.2948
   1.2299
   2.0066
   1.5930
   1.3400
   1.4915
   1.0886
   1.9057
   1.6553
   1.4229
   1.2962
   1.3107
   1.3717
   1.2942
   1.7161
   1.8369
   1.7152
   1.7419
   1.7757
   1.9737
   1.3187
   1.2897
   1.2364
   1.7114
   1.9212
   1.7289
   1.2250
   1.8458
   1.2522
   1.9403
   1.4151
   1.1191
   1.8523
   1.9647
   1.9493
   1.9043
   1.8525
   1.5948
   1.3247
   1.5095
   1.2233

ttb =  310.91
Optimizing qs: remain iter - 4
qs =

   2.0514
   1.9170
   1.6218
   1.3928
   1.2046
   1.9137
   1.8090
   1.8

In [12]:
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
qs
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   2.0635
   1.9172
   1.6218
   1.3928
   1.2046
   1.9138
   1.8099
   1.8132
   1.3535
   1.2977
   1.2299
   2.0083
   1.5953
   1.3400
   1.4960
   1.0886
   1.9057
   1.6567
   1.4232
   1.2962
   1.3107
   1.3717
   1.2942
   1.7161
   1.8369
   1.7152
   1.7433
   1.7757
   1.9737
   1.3190
   1.3354
   1.2378
   1.7114
   1.9212
   1.7289
   1.2205
   1.8459
   1.2541
   1.9403
   1.4174
   1.1215
   1.8523
   1.9647
   1.9493
   1.9046
   1.8546
   1.5948
   1.3272
   1.5095
   1.2294

TERBCM: SMSE 38.7455, MSLL -0.8582, NLPD 3.3602


In [13]:
save('./res/videoqs', 'qs');
save('./res/video_res', 'SMSE','MSLL','NLPD')